In [1]:
#link
#https://github.com/jxzly/Biomedical-Relation-Classification/tree/b74ce58a6a47af2cffdcaea2949ad359bf75b07b
!pip install pyecharts

You should consider upgrading via the 'c:\users\marija\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
import numpy as np 
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from pyecharts import options as opts
from pyecharts.charts import Graph
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset,TensorDataset,DataLoader
from keras.preprocessing import sequence
from transformers import BertTokenizer, BertForSequenceClassification

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [ ]:
from torch.utils.data import Dataset,TensorDataset,DataLoader

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

class Conf:
    # some information can be found in:
    # Percha B, Altman R B. A global network of biomedical relationships derived from text[J]. Bioinformatics, 2018, 34(15): 2614-2624.
    download_url = {'chemical-disease':['https://zenodo.org/record/1035500/files/part-i-chemical-disease-path-theme-distributions.txt?download=1',
                                             'https://zenodo.org/record/1035500/files/part-ii-dependency-paths-chemical-disease-sorted-with-themes.txt?download=1'],
                    'chemical-gene':['https://zenodo.org/record/1035500/files/part-i-chemical-gene-path-theme-distributions.txt?download=1',
                                                               'https://zenodo.org/record/1035500/files/part-ii-dependency-paths-chemical-gene-sorted-with-themes.txt?download=1'],
                    'gene-disease':['https://zenodo.org/record/1035500/files/part-i-gene-disease-path-theme-distributions.txt?download=1',
                                             'https://zenodo.org/record/1035500/files/part-ii-dependency-paths-gene-disease-sorted-with-themes.txt?download=1'],
                    }
    usecols = {'chemical-disease':['path', 'T', 'C', 'Sa', 'Pr', 'Pa', 'J', 'Mp'],
               'chemical-gene':['path', 'A+', 'A-', 'B', 'E+', 'E-', 'E', 'N', 'O', 'K', 'Z'],
               'gene-disease':['path', 'U', 'Ud', 'D', 'J', 'Te', 'Y', 'G', 'Md', 'X', 'L'],
               'gene-gene':['path', 'B', 'W', 'V+', 'E+', 'E', 'I', 'H', 'Rg', 'Q'],
               }
    relation_type = {'chemical-disease':['T', 'C', 'Sa', 'Pr', 'Pa', 'J'],
                     'disease-chemical':['Mp'],
                     'chemical-gene':['A+', 'A-', 'B', 'E+', 'E-', 'E', 'N'],
                     'gene-chemical':['O', 'K', 'Z'],
                     'gene-disease':['U', 'Ud', 'D', 'J', 'Te', 'Y', 'G'],
                     'disease-gene':['Md', 'X', 'L'],
                     'gene-gene':['B', 'W', 'V+', 'E+', 'E', 'I', 'H', 'Rg', 'Q']
                     }
    relation_theme = {'chemical-disease':{'T':'Treatment/therapy (incl. investigatory)',
                                          'C':'Inhibits cell growth (esp. cancers)',
                                          'Sa':'Side effect/adverse event',
                                          'Pr':'Prevents, suppresses',
                                          'Pa':'Alleviates, reduces',
                                          'J':'Role in pathogenesis',
                                          },
                      'disease-chemical':{'Mp':'Biomarkers (progression)',
                                          },
                      'chemical-gene':{'A+':'Agonism, activation',
                                       'A-':'Antagonism, blocking',
                                       'B':'Binding, ligand (esp. receptors)',
                                       'E+':'Increases expression/production',
                                       'E-':'Decreases expression/production',
                                       'E':'Affects expression/production (neutral)',
                                       'N':'Inhibits',
                                       },
                      'gene-chemical':{'O':'Transport, channels',
                                       'K':'Metabolism, pharmacokinetics',
                                       'Z':'Enzyme activity',
                                       },
                      'gene-disease':{'U':'Causal mutations',
                                      'Ud':'Mutations affect disease course',
                                      'D':'Drug targets',
                                      'J':'Role in pathogenesis',
                                      'Te':'Possible therapeutic effect',
                                      'Y':'Polymorphisms alter risk',
                                      'G':'Promotes progression',
                                      },
                      'disease-gene':{'Md':'Biomarkers (diagnostic)',
                                      'X':'Overexpression in disease',
                                      'L':'Improper regulation linked to disease',
                                      },
                      'gene-gene':{'B':'Binding, ligand (esp. receptors)',
                                   'W':'Enhances response',
                                   'V+':'Activates, stimulates',
                                   'E+':'Increases expression/production',
                                   'E':'Affects expression/production (neutral)',
                                   'I':'Signaling pathway',
                                   'H':'Same protein or complex',
                                   'Rg':'Regulation',
                                   'Q':'Production by cell population',
                                   }
                       }
    confidence_limit = {'chemical-disease':0.9,
                        'chemical-gene':0.5,
                        'gene-disease':0.6,
                        'gene-gene':0.9,
                        }

def Data_loader(x,y=None,bs=128,shuffle=False,numWorkers=0):
    if y is not None:
        data = TensorDataset(x,y)
    else:
        data = TensorDataset(x)
    data_loader = DataLoader(dataset=data,batch_size=bs,shuffle=shuffle,num_workers=numWorkers)
    return data_loader

def Bert_conf(tokenizer=None,model=None):
    # add some tokens to vocab
    if tokenizer != None:
        num_added_toks = tokenizer.add_tokens(['start_entity', 'end_entity'])
        print('We have added', num_added_toks, 'tokens')
    if model != None:
        model.resize_token_embeddings(len(tokenizer))
    return None

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import BertTokenizer, BertForSequenceClassification


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

def Bert_model(taskType,bertPath):
    label_df = pd.read_csv('D:\\IJS_MASTER\\SemanticWebTechnologies\\Covid19_Dataset\\%s_label.csv'%taskType)
    tokenizer = BertTokenizer.from_pretrained(bertPath,do_lower_case=False)
    model = BertForSequenceClassification.from_pretrained(bertPath, num_labels=label_df['label'].nunique())
    Bert_conf(tokenizer,model)
    return tokenizer,model

In [ ]:
import numpy as np
import pandas as pd
import sys,os,re
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import torch
from keras.preprocessing import sequence


import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

class Prepare_Data(Conf):
    def __init__(self,taskType,confidenceLimit,predictionPath,maxSeqLen,bertPath):
        super(Prepare_Data,self).__init__()
        self.task_type = taskType
        self.confidence_limit = confidenceLimit
        self.prediction_path = predictionPath
        self.max_seq_len = maxSeqLen
        self.bert_path = bertPath

    def Prepare_train_test_data(self,tokenizer,trainBS,evalBS):
        # if not exist original data, download them
    

        # build data loder
        path_theme_df = pd.read_csv('D:\\IJS_MASTER\\SemanticWebTechnologies\\Covid19_Dataset\\data\\part-i-%s-path-theme-distributions.txt'%self.task_type,sep='\t',
                                    usecols=self.usecols[self.task_type])
        path_theme_df['max_value_prob'] = path_theme_df[self.usecols[self.task_type][1:]].max(axis=1) / path_theme_df[self.usecols[self.task_type][1:]].sum(axis=1)
        path_theme_df = path_theme_df.loc[path_theme_df['max_value_prob']>self.confidence_limit].reset_index(drop=True)
        path_theme_df['label_raw'] = path_theme_df[self.usecols[self.task_type][1:]].apply(lambda x:x.argmax(),axis=1)
        path_theme_df.drop(self.usecols[self.task_type][1:],axis=1,inplace=True)
        sentence_df = pd.read_csv('D:\\IJS_MASTER\\SemanticWebTechnologies\\Covid19_Dataset\\data\\part-ii-dependency-paths-%s-sorted-with-themes.txt'%self.task_type,sep='\t',header=None,
                                  names=['PubMed_ID','sentence_number','start_entity','start_entity_location','end_entity','end_entity_location','start_entity_raw','end_entity_raw','start_entity_database_id','end_entity_database_id','start_entity_type','end_entity_type','path','sentence_tokenized'],
                                  usecols=['start_entity','end_entity','path','sentence_tokenized'],nrows=None)
        sentence_df = sentence_df.loc[~((sentence_df['start_entity'].isna())|(sentence_df['end_entity'].isna()))].reset_index(drop=True)
        sentence_df['path'] = sentence_df['path'].apply(lambda x:x.lower())
        sentence_df['sentence_tokenized'] = sentence_df.apply(lambda x:x['sentence_tokenized'].replace(x['start_entity'],'start_entity').replace(x['end_entity'],'end_entity'),axis=1)
        sentence_df = sentence_df.merge(path_theme_df[['path','label_raw']],how='inner',on='path')

        # encoder labels
        le = preprocessing.LabelEncoder()
        le.fit(sentence_df['label_raw'].values.tolist())
        label_df = pd.DataFrame({'label_raw':le.classes_,'label':list(range(len(le.classes_)))})
        # map label to theme
        theme_dic = self.relation_theme[self.task_type]
        if self.task_type.split('-')[0] != self.task_type.split('-')[1]:
            theme_dic.update(self.relation_theme[self.task_type.split('-')[1]+'-'+self.task_type.split('-')[0]])
        label_df['theme'] = label_df['label_raw'].map(theme_dic)
        label_df.to_csv('D:\\IJS_MASTER\\SemanticWebTechnologies\\Covid19_Dataset\\data\\%s_label.csv'%self.task_type,index=False)
        sentence_df['label'] = le.transform(sentence_df['label_raw'].values)

        # covert sentences to ids in bert
        ids = sentence_df['sentence_tokenized'].apply(lambda x:tokenizer.convert_tokens_to_ids(tokenizer.tokenize(x))).tolist()
        ids = sequence.pad_sequences(ids,self.max_seq_len, truncating='post', padding='post')

        # split data to train and test
        X_train, X_test, y_train, y_test = train_test_split(ids,sentence_df['label'].values, test_size=0.2, stratify=sentence_df['label'].values, random_state=2020)
        train_data_loader = Data_loader(torch.LongTensor(X_train),torch.LongTensor(y_train),bs=trainBS)
        test_data_loader = Data_loader(torch.LongTensor(X_test),torch.LongTensor(y_test),bs=evalBS)
        return train_data_loader,test_data_loader

    def Prepare_predict_data(self,tokenizer,bs):
        #tuka proveri
        marked_sentence_df = pd.read_csv('D:\\IJS_MASTER\\SemanticWebTechnologies\\Covid19_Dataset\\marked_sentence_2.csv')
        marked_sentences = marked_sentence_df.loc[(marked_sentence_df['start_entity_type'].apply(lambda x:x.lower())==self.task_type.split('-')[0])&\
                                                  (marked_sentence_df['end_entity_type'].apply(lambda x:x.lower())==self.task_type.split('-')[1]),'marked_sentence']

        ids = marked_sentences.apply(lambda x:tokenizer.convert_tokens_to_ids(tokenizer.tokenize(x))).tolist()
        ids = sequence.pad_sequences(ids,self.max_seq_len, truncating='post', padding='post')
        # we cannot confirm order of entities, so predict two possibilities
        reverse_marked_sentences = marked_sentence_df.loc[(marked_sentence_df['start_entity_type'].apply(lambda x:x.lower())==self.task_type.split('-')[0])&\
                                                  (marked_sentence_df['end_entity_type'].apply(lambda x:x.lower())==self.task_type.split('-')[1]),'marked_sentence']\
                                                  .apply(lambda x:x.replace('start_entity','init_start_entity').replace('end_entity','start_entity').replace('init_start_entity','end_entity'))
        reverse_ids = reverse_marked_sentences.apply(lambda x:tokenizer.convert_tokens_to_ids(tokenizer.tokenize(x))).tolist()
        reverse_ids = sequence.pad_sequences(reverse_ids,self.max_seq_len, truncating='post', padding='post')
        predict_data_loader = Data_loader(torch.LongTensor(ids),torch.LongTensor(reverse_ids),bs=bs)
        return marked_sentences.values,predict_data_loader

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import f1_score
import torch
import torch.nn.functional as F
from transformers import BertTokenizer
import argparse

conf = Conf()

def Parse_args():
    args = argparse.ArgumentParser()
    args.add_argument('--task_type',
                      default='chemical-disease', help='task type:chemical-disease,chemical-gene,gene-disease')
    args.add_argument('--confidence_limit', type=float,
                      default=-1.0, help='dependency path lower confidence limit, use suggestion value if it equal -1.0. \
                      suggestion value:0.9 for chemical-disease; 0.5 for chemical-gene; 0.6 for gene-disease; 0.9 for gene-gene')
    args.add_argument('--prediction_path',
                      default='COVID-19', help='prediction data path')
    args.add_argument('--max_seq_len', type=int,
                      default=64, help='padding length of sequence')
    args.add_argument('--bert_path',
                      default='./pretrained/bert-base-cased', help='bert model path')
    args.add_argument('--lr', type=float, default=1e-5)
    args.add_argument('--train_bs', type=int, default=128, help='train batch size')
    args.add_argument('--eval_bs', type=int, default=128, help='evaluate batch size')
    args.add_argument('--epochs', type=int, default=100)
    args.add_argument('--cuda', type=int, default=0, help='which gpu be used')
    args = args.parse_args()
    return args

args = Parse_args()
# dependency path lower confidence limit, use suggestion value if it equal -1.0
confidence_limit = args.confidence_limit if args.confidence_limit != -1.0 else conf.confidence_limit[args.task_type]
tokenizer = BertTokenizer.from_pretrained(args.bert_path,do_lower_case=False)
Bert_conf(tokenizer)
prepare = Prepare_Data(args.task_type,confidence_limit,args.prediction_path,args.max_seq_len,args.bert_path)

def Load_train_test_data():
    train_data_loader,test_data_loader = prepare.Prepare_train_test_data(tokenizer,args.train_bs,args.eval_bs)
    return train_data_loader,test_data_loader

def Load_predict_data():
    marked_sentences,predict_data_loader = prepare.Prepare_predict_data(tokenizer,args.eval_bs)
    return marked_sentences,predict_data_loader

train_data_loader,test_data_loader = Load_train_test_data()
marked_sentences,predict_data_loader = Load_predict_data()
device = torch.device('cuda:%s'%args.cuda if torch.cuda.is_available() else 'cpu')
print('device:', device)

def Train(evalEpochs=None):
    tokenizer,model = Bert_model(args.task_type,args.bert_path)
    tokenizer.save_pretrained('D:\\IJS_MASTER\\SemanticWebTechnologies\\Biomedical-Relation-Classification-master\\model\\%s'%args.task_type)
    model = model.to(device)
    model_params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.Adam(model_params, lr=args.lr)
    for epoch in range(args.epochs):
        running_loss = 0.0
        for data in tqdm(train_data_loader):
            ids, labels = [t.to(device) for t in data]
            optimizer.zero_grad()
            # forward pass
            outputs = model(input_ids=ids,labels=labels)
            loss = outputs[0]
            # backward
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'[epoch {epoch+1}] loss: {running_loss:3f}')
        if evalEpochs != None:
            if (epoch+1)%evalEpochs == 0:
                torch.cuda.empty_cache()
                Evaluate(model)
    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
    model_to_save.save_pretrained('D:\\IJS_MASTER\\SemanticWebTechnologies\\Biomedical-Relation-Classification-master\\model\\%s'%args.task_type)
    torch.cuda.empty_cache()
    return

def Evaluate(model=None):
    if model == None:
        tokenizer,model = Bert_model(args.task_type,'D:\\IJS_MASTER\\SemanticWebTechnologies\\Biomedical-Relation-Classification-master\\model\\%s'%args.task_type)
        model = model.to(device)
    test_preds,test_labels = [],[]
    for data in tqdm(test_data_loader):
        ids, labels = [t.to(device) for t in data]
        outputs = model(input_ids=ids)
        logits = outputs[0]
        _, pred = torch.max(logits.data, 1)
        test_preds.extend(list(pred.cpu().detach().numpy()))
        test_labels.extend(list(labels.cpu().detach().numpy()))
    macro_f1 = f1_score(test_labels,test_preds,average='macro')
    print('test macro f1 score:%.4f'%macro_f1)
    torch.cuda.empty_cache()
    return

def Predict():
    reverse_task_type = args.task_type.split('-')[1] + '-' + args.task_type.split('-')[0]
    def Filter(x):
        if x['init_pred'] in conf.relation_type[args.task_type]:
            if x['reverse_pred'] not in conf.relation_type[reverse_task_type]:
                # init_pred is a correct relation but reverse_pred not
                return 'init_pred'
            else:
                # init_pred and reverse_pred both are correct relations
                if x['init_pred_prob'] >= x['reverse_pred_prob']:
                    # init_pred_prob greater than or equal to reverse_pred_prob
                    return 'init_pred'
                else:
                    return 'reverse_pred'
        else:
            if x['reverse_pred'] not in conf.relation_type[reverse_task_type]:
                # init_pred and reverse_pred both are uncorrect relations
                return 'uncorrect'
            else:
                # reverse_pred is a correct relation but init_pred not
                return 'reverse_pred'
    marked_sentence_df = pd.read_csv('./data/%s/marked_sentence.csv'%args.prediction_path)
    label_df = pd.read_csv('D:\\IJS_MASTER\\SemanticWebTechnologies\\Covid19_Dataset\\%s_label.csv'%args.task_type)
    tokenizer,model = Bert_model(args.task_type,'D:\\IJS_MASTER\\SemanticWebTechnologies\\Biomedical-Relation-Classification-master\\model\\%s'%args.task_type)
    model = model.to(device)
    preds = []
    preds_prob = []
    reverse_preds = []
    reverse_preds_prob = []
    for data in tqdm(predict_data_loader):
        ids,reverse_ids = [t.to(device) for t in data]
        outputs = model(input_ids=ids)
        logits = outputs[0]
        pred_prob, pred = torch.max(F.softmax(logits.data,1), 1)
        preds.extend(list(pred.cpu().detach().numpy()))
        preds_prob.extend(list(pred_prob.cpu().detach().numpy()))
        reverse_outputs = model(input_ids=reverse_ids)
        reverse_logits = reverse_outputs[0]
        reverse_pred_prob, reverse_pred = torch.max(F.softmax(reverse_logits.data,1), 1)
        reverse_preds.extend(list(reverse_pred.cpu().detach().numpy()))
        reverse_preds_prob.extend(list(reverse_pred_prob.cpu().detach().numpy()))

    pred_df = pd.DataFrame({'marked_sentence':marked_sentences,'init_pred':preds,'init_pred_prob':preds_prob,'reverse_pred':reverse_preds,'reverse_pred_prob':reverse_preds_prob})
    # map label(0, 1, 2...) to raw label(T, C, Sa...)
    pred_df['init_pred'] = pred_df['init_pred'].replace(dict(label_df.set_index(['label'])['label_raw']))
    pred_df['reverse_pred'] = pred_df['reverse_pred'].replace(dict(label_df.set_index(['label'])['label_raw']))
    # judge the order of a pair of entities
    pred_df['filter'] = pred_df.apply(lambda x:Filter(x), axis=1)
    pred_df['pred'] = pred_df['init_pred']
    pred_df['pred_prob'] = pred_df['init_pred_prob']
    pred_df.loc[pred_df['filter']=='reverse_pred','pred'] = pred_df.loc[pred_df['filter']=='reverse_pred','reverse_pred']
    pred_df.loc[pred_df['filter']=='reverse_pred','pred_prob'] = pred_df.loc[pred_df['filter']=='reverse_pred','reverse_pred_prob']
    pred_df = pred_df.loc[pred_df['filter']!='uncorrect']
    pred_df = marked_sentence_df.merge(pred_df,how='inner',on='marked_sentence')
    pred_df['init_start_entity'] = pred_df['start_entity']
    pred_df['init_start_entity_type'] = pred_df['start_entity_type']
    pred_df.loc[pred_df['filter']=='reverse_pred','start_entity'] = pred_df.loc[pred_df['filter']=='reverse_pred','end_entity']
    pred_df.loc[pred_df['filter']=='reverse_pred','start_entity_type'] = pred_df.loc[pred_df['filter']=='reverse_pred','end_entity_type']
    pred_df.loc[pred_df['filter']=='reverse_pred','end_entity'] = pred_df.loc[pred_df['filter']=='reverse_pred','init_start_entity']
    pred_df.loc[pred_df['filter']=='reverse_pred','end_entity_type'] = pred_df.loc[pred_df['filter']=='reverse_pred','init_start_entity_type']
    pred_df.drop(['init_start_entity','init_start_entity_type'],axis=1,inplace=True)
    pred_df.to_csv('D:\\IJS_MASTER\\SemanticWebTechnologies\\Covid19_Dataset\\data\\%s\\%s_pred.csv'%(args.prediction_path,args.task_type),index=False)
    torch.cuda.empty_cache()
    return

Train(evalEpochs=5)
Evaluate()


In [ ]:
def Build_graph(df,relation=False,repulsion=40,title='COVID-19 knowledge graph',labelShow=False):
    entity_type_dic = dict(df.drop_duplicates(['start_entity']).set_index(['start_entity'])['start_entity_type'])
    entity_type_dic.update(dict(df.drop_duplicates(['end_entity']).set_index(['end_entity'])['end_entity_type']))
    color = {'Disease':'#FF7F50','Gene':'#48D1CC','Chemical':'#B3EE3A'}
    cate =  {'Disease':0,'Gene':1,'Chemical':2}
    categories = [{'name':'Disease','itemStyle': {'normal': {'color': color['Disease']}}},{'name':'Gene','itemStyle': {'normal': {'color': color['Gene']}}},{'name':'Chemical','itemStyle': {'normal': {'color': color['Chemical']}}}]
    nodes = []
    for entity in list(set(df['start_entity'])|set(df['end_entity'])):
        nodes.append({'name': entity, 'symbolSize': max(10,np.log1p(df.loc[(df['start_entity']==entity)|(df['end_entity']==entity)].shape[0])*10//1),
                     'category':cate[entity_type_dic[entity]]})
    links = []
    for i in df.index:
        if not relation:
            links.append({'source': df.loc[i,'start_entity'], 'target': df.loc[i,'end_entity']})
        else:
            links.append({'source': df.loc[i,'start_entity'], 'target': df.loc[i,'end_entity'], 'value':df.loc[i,'pred']})
    g = (
        Graph()
        .add('', nodes, links,categories, repulsion=repulsion,label_opts=opts.LabelOpts(is_show=labelShow))
        .set_global_opts(title_opts=opts.TitleOpts(title=title),legend_opts=opts.LegendOpts(orient='vertical', pos_left='2%', pos_top='40%',legend_icon='circle'))
        .render_notebook()
        )
    return g

In [ ]:
args.task_type = 'chemical-disease'
c_d_label_df,c_d_pred_df = Predict()

In [ ]:
# chemical-disease relation theme
c_d_label_df

In [ ]:
# chemical-disease classification results
c_d_pred_df.sample(5)

In [ ]:
# chemical-gene relation prediction
args.task_type = 'chemical-gene'
c_g_label_df,c_g_pred_df = Predict()

In [ ]:
# chemical-gene relation theme
c_g_label_df

In [ ]:
# chemical-gene classification results
c_g_pred_df.sample(5)

In [ ]:
# gene-disease relation prediction
args.task_type = 'gene-disease'
g_d_label_df,g_d_pred_df = Predict()

In [ ]:
# gene-disease relation theme
g_d_label_df

In [ ]:
# gene-disease classification results
g_d_pred_df.sample(5)

In [ ]:
# gene-gene relation prediction
args.task_type = 'gene-gene'
g_g_label_df,g_g_pred_df = Predict()

In [ ]:
# gene-gene relation theme
g_g_label_df

In [ ]:
# gene-gene classification results
g_g_pred_df.sample(5)

In [ ]:
# gene-gene relation theme
g_g_label_df

In [ ]:
# gene-gene classification results
g_g_pred_df.sample(5)

In [ ]:
# chemicl-COVID-19 relations
g = Build_graph(c_d_pred_df.loc[(c_d_pred_df['start_entity']=='COVID-19')|(c_d_pred_df['end_entity']=='COVID-19')],relation=True,repulsion=800,title='chemical-COVID-19 knowledge graph',labelShow=True)
g

In [ ]:
# gene-COVID-19 relations
g = Build_graph(g_d_pred_df.loc[(g_d_pred_df['start_entity']=='COVID-19')|(g_d_pred_df['end_entity']=='COVID-19')],relation=True,repulsion=60,title='gene-COVID-19 knowledge graph',labelShow=False)
g

In [ ]:
# gene-COVID-19 relations
g = Build_graph(marked_sentence_df.loc[(marked_sentence_df['start_entity']=='COVID-19')&(marked_sentence_df['end_entity_type']=='Disease')|(marked_sentence_df['start_entity_type']=='Disease')&(marked_sentence_df['end_entity']=='COVID-19')],relation=False,repulsion=60,title='disease-COVID-19 topology graph',labelShow=False)
g

In [ ]:
# merge all relation prediction and save results
cols = ['start_entity','end_entity','start_entity_type','end_entity_type','marked_sentence','pred','pred_prob']
relation_df = pd.concat([c_d_pred_df[cols],c_g_pred_df[cols],g_d_pred_df[cols],g_g_pred_df[cols]]).append(marked_sentence_df.loc[(marked_sentence_df['start_entity_type'].isin(['Chemical','Disease'])&(marked_sentence_df['start_entity_type']==marked_sentence_df['end_entity_type']))]).reset_index(drop=True)
relation_df.loc[(relation_df['pred'].isna())&(relation_df['start_entity_type']=='Chemical'),'pred'] = 'CC'
relation_df.loc[(relation_df['pred'].isna())&(relation_df['start_entity_type']=='Disease'),'pred'] = 'DD'
relation_df = relation_df[cols]
relation_df.to_csv('relation.csv',index=False)

In [ ]:
relation_df

In [ ]:
# subsample of knowledge graph
g = Build_graph(relation_df.sample(1000),relation=True,repulsion=15,title='subsample of COVID-19 knowledge graph',labelShow=False)
g